Credits:

* https://github.com/pmbaumgartner/setfit
* https://moshewasserblat.medium.com/sentence-transformer-fine-tuning-setfit-outperforms-gpt-3-on-few-shot-text-classification-while-d9a3788f0b4e

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nudges-001/Final_data1.csv
/kaggle/input/nudges-02/nudges_02.csv
/kaggle/input/nudges-02/nudges_02 (1).csv
/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


In [14]:
df=pd.read_csv('/kaggle/input/nudges-001/Final_data1.csv')
df.head()
#df = df.drop(['Unnamed: 0'],axis=1)

,Theme,Nudges
0,Water Saved,Adopt cultivation of less water intensive crop...
1,Water Saved,Adopt cultivation of less water intensive crop...
2,Water Saved,Adopt cultivation of less water intensive crop...
3,Water Saved,Adopt cultivation of less water intensive crop...
4,Water Saved,Adopt cultivation of less water intensive crop...


In [15]:
df.describe()
df = df.drop_duplicates()


In [16]:
df.Theme.value_counts()

Energy Saved                  88
Water Saved                   72
LiFEStyles Adopted            61
Air Quality                   49
Single-Use Plastic Reduced    40
Name: Theme, dtype: int64

In [17]:
#df = df.loc[df['Theme'] != 'Sustainable Food Systems Adopted']
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# df['Theme_numeric'] = label_encoder.fit_transform(df['Theme'])


In [18]:
df

,Theme,Nudges
0,Water Saved,Adopt cultivation of less water intensive crop...
1,Water Saved,Adopt cultivation of less water intensive crop...
3,Water Saved,Adopt cultivation of less water intensive crop...
5,Water Saved,Adopt cultivation of less water intensive crop...
9,Water Saved,Adopt cultivation of less water intensive crop...
...,...,...
721,Air Quality,Urban Forestation Programs
722,Air Quality,Environmentally Conscious Consumer Choices
723,Air Quality,Public Bicycle Sharing Programs
724,Air Quality,Green Roofs and Walls


In [19]:
df.head(30)

,Theme,Nudges
0,Water Saved,Adopt cultivation of less water intensive crop...
1,Water Saved,Adopt cultivation of less water intensive crop...
3,Water Saved,Adopt cultivation of less water intensive crop...
5,Water Saved,Adopt cultivation of less water intensive crop...
9,Water Saved,Adopt cultivation of less water intensive crop...
10,Water Saved,Adopt cultivation of less water intensive crop...
12,Water Saved,Adopt cultivation of less water intensive crop...
15,Water Saved,Adopt cultivation of less water intensive crop...
23,Water Saved,Adopt cultivation of less water intensive crop...
30,Water Saved,Adopt cultivation of less water intensive crop...


In [20]:
df.Theme.value_counts()

Energy Saved                  88
Water Saved                   72
LiFEStyles Adopted            61
Air Quality                   49
Single-Use Plastic Reduced    40
Name: Theme, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Theme'])  # Features
y = df['Theme']  # Target variable




In [22]:
# Use train_test_split with stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (217, 1)
X_test shape: (93, 1)
y_train shape: (217,)
y_test shape: (93,)


In [23]:
X_train

,Nudges
715,Encouraging Low Emission Construction Equipment
704,Sensitizing Industries on Pollution Prevention
169,Initiate and/or join green clubs in your resid...
41,Create rainwater harvesting infrastructure in ...
36,Carry your own water bottle wherever possible ...
...,...
514,Use LED bulbs/ tube-lights test test test test...
168,Initiate and/or join green clubs in your resid...
288,Pre-soak heavy pots and pans before washing th...
686,Waste Reduction and Recycling Awareness


In [24]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.utils import shuffle

def augment_data_to_target(X, y, target_samples_per_class):
    """
    Augment the data to achieve the target number of samples for each class.

    Parameters:
    - X: Feature matrix
    - y: Target variable
    - target_samples_per_class: Number of samples to generate for each class

    Returns:
    - X_augmented: Augmented feature matrix
    - y_augmented: Augmented target variable
    """
    unique_classes, class_counts = np.unique(y, return_counts=True)

    # Create a dictionary with the target number of samples for each class
    samples_to_generate = {cls: target_samples_per_class for cls in unique_classes}

    # Create a pipeline with RandomOverSampler and any other necessary preprocessing steps
    pipeline = Pipeline([
        ('oversampler', RandomOverSampler(sampling_strategy=samples_to_generate)),
        # Add other preprocessing steps if needed
    ])

    # Fit the pipeline on the entire dataset
    X_resampled, y_resampled = pipeline.fit_resample(X, y)

    # Shuffle the augmented data
    X_augmented, y_augmented = shuffle(X_resampled, y_resampled, random_state=42)

    return X_augmented, y_augmented

# Assuming X_train and y_train are your original training data
X_train_augmented, y_train_augmented = augment_data_to_target(X_train, y_train, target_samples_per_class=100)


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (88) in class Air Quality will be larger than the number of samples in the majority class (class #Energy Saved -> 54)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (88) in class Energy Saved will be larger than the number of samples in the majority class (class #Energy Saved -> 54)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (88) in class LiFEStyles Adopted will be larger than the number of samples in the majority class (class #Energy Saved -> 54)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validat

In [25]:
y_train_augmented.value_counts()

Air Quality                   88
Water Saved                   88
Single-Use Plastic Reduced    88
LiFEStyles Adopted            88
Energy Saved                  88
Name: Theme, dtype: int64

In [21]:
!pip install git+https://github.com/pmbaumgartner/setfit -q

In [24]:
from setfit import SetFitClassifier

In [28]:
# takes a sentence-transformers model
clf = SetFitClassifier("paraphrase-MiniLM-L3-v2")

In [29]:
docs = X_train_augmented['Nudges'].values.tolist()
labels = y_train_augmented


In [30]:
# fine-tunes embeddings + trains logistic regression head
clf.fit(docs, labels)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/275 [00:00<?, ?it/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [31]:
docs_test = X_test['Nudges'].values.tolist()
labels_test = y_test
y_pred = clf.predict(docs_test)


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)



Accuracy: 0.99
Classification Report:
                             precision    recall  f1-score   support

               Air Quality       1.00      1.00      1.00        11
              Energy Saved       1.00      0.97      0.99        34
        LiFEStyles Adopted       1.00      1.00      1.00        14
Single-Use Plastic Reduced       1.00      1.00      1.00        11
               Water Saved       0.96      1.00      0.98        23

                  accuracy                           0.99        93
                 macro avg       0.99      0.99      0.99        93
              weighted avg       0.99      0.99      0.99        93



In [ ]:
# Test the model with a single input text
input_text = "use jute bags"
predicted_category = clf.predict([input_text])[0]
print(f"Predicted Category for the input text: {predicted_category}")

# PREDICTION

In [57]:
def predict_top_classes(input_text, clf, num_classes=3):
    # Predict probabilities for each class
    probabilities = clf.predict_proba([input_text])[0]

    # Get indices of the top N classes
    top_indices = np.argsort(probabilities)[-num_classes:][::-1]

    # Get class names and corresponding probabilities
    top_classes = []
    for index in top_indices:
        probability = probabilities[index]
        top_classes.append((index, probability))

    return top_classes

# Assuming you have already trained the classifier clf
input_text = "jutes bags reuse moisture"
top_classes = predict_top_classes(input_text, clf, num_classes=3)

print(f"Top classes for '{input_text}':")
for class_index, probability in top_classes:
    print(f"Class: {class_index}, Probability: {probability:.4f}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top classes for 'jutes bags reuse moisture':
Class: 3, Probability: 0.9342
Class: 0, Probability: 0.0353
Class: 4, Probability: 0.0293


In [62]:
df.head(40)

,Theme,Nudges
0,Water Saved,Adopt cultivation of less water intensive crop...
1,Water Saved,Adopt cultivation of less water intensive crop...
3,Water Saved,Adopt cultivation of less water intensive crop...
5,Water Saved,Adopt cultivation of less water intensive crop...
9,Water Saved,Adopt cultivation of less water intensive crop...
10,Water Saved,Adopt cultivation of less water intensive crop...
12,Water Saved,Adopt cultivation of less water intensive crop...
15,Water Saved,Adopt cultivation of less water intensive crop...
23,Water Saved,Adopt cultivation of less water intensive crop...
30,Water Saved,Adopt cultivation of less water intensive crop...


In [65]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    text = ' '.join([word for word in word_tokens if word not in stop_words])
    
    return text

# Apply the cleaning function to the 'Brief Description of Activity' column
df['Nudges'] = df['Nudges'].apply(clean_text)


In [66]:
df.sort_values(by='Theme', inplace=True)
df.to_csv("final_001.csv")

# PARAMETERS

In [2]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
df = pd.read_csv("/kaggle/input/air-parameters/air_parameters.csv")

In [4]:


# Split the indicators into a list of strings
df['Indicators Affected'] = df['Indicators Affected'].str.split(', ')

# Create binary labels using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(df['Indicators Affected'])
labels_df = pd.DataFrame(binary_labels, columns=mlb.classes_)

# Concatenate the binary labels with the original DataFrame
df = pd.concat([df['Behavioral Nudge'], labels_df], axis=1)

# Save the processed data to a new CSV file or use it for training directly
df.to_csv('multilabel_data_sklearn.csv', index=False)

# Display the processed DataFrame
print(df)

                                     Behavioral Nudge  Benzene  CO  NH3  NO2  \
0                 Eco-friendly Firecrackers on Diwali        1   1    1    1   
1                      Promotion of Electric Vehicles        1   1    1    1   
2     Public Awareness Campaigns on Vehicle Emissions        1   1    1    1   
3             Incentives for Green Building Practices        1   1    1    1   
4        Tree Plantation and Green Spaces Development        1   1    1    1   
5              Sustainable Transportation Initiatives        1   1    1    1   
6               Carpooling and Ride-sharing Promotion        1   1    1    1   
7                   Advocacy for Clean Energy Sources        1   1    1    1   
8                   Reducing Idling Time for Vehicles        1   1    1    1   
9             Waste Reduction and Recycling Awareness        1   1    1    1   
10                               Smart Urban Planning        1   1    1    1   
11                  Eco-friendly Industr

In [6]:
!pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 10.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=87874b5575a4336b77206ba7479fc0f58497605b904c92ca3a5c4bf91a97a598
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.7.0
    Uninstalling huggingface-hub-0.7.0:
      Successfully uninstalled huggingface-hub-0.7.0
  Attempting uninstall: datasets
  